<a href="https://colab.research.google.com/github/Varsha-code-bit/INTERVIEW-QUESTIONS/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wolta
!pip install ultralytics
!pip install transformers
!pip install torch torchvision
!pip install tensorflow
!pip install hyperopt

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import cv2
import os
from glob import glob
from pathlib import Path
import zipfile

from wolta.visual_tools import get_extensions, dataset_size_same, dataset_ratio_same, examine_sizes, dir_split
from ultralytics import YOLO
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification
from torch import nn
from torch.optim import Adam
from tensorflow.keras import layers, Sequential

os.environ['WANDB_MODE'] = 'disabled'


In [ ]:
# Paths
zip_path = '/content/dataset_ISL_.zip'
extract_path = '/content/isl_dataset'

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Files extracted to:", extract_path)

# Check structure
for dirname, _, _ in os.walk(extract_path):
    print(dirname)


Files extracted to: /content/isl_dataset
/content/isl_dataset
/content/isl_dataset/o
/content/isl_dataset/d
/content/isl_dataset/v
/content/isl_dataset/s
/content/isl_dataset/a
/content/isl_dataset/m
/content/isl_dataset/y
/content/isl_dataset/k
/content/isl_dataset/u
/content/isl_dataset/h
/content/isl_dataset/e
/content/isl_dataset/f
/content/isl_dataset/n
/content/isl_dataset/t
/content/isl_dataset/q
/content/isl_dataset/l
/content/isl_dataset/j
/content/isl_dataset/x
/content/isl_dataset/r
/content/isl_dataset/p
/content/isl_dataset/w
/content/isl_dataset/i
/content/isl_dataset/b
/content/isl_dataset/g
/content/isl_dataset/c
/content/isl_dataset/z


In [ ]:
from glob import glob
import os

extract_path = '/content/isl_dataset'  # Ensure extract_path is defined

# Use glob to find all files within the extracted directory and its subdirectories
all_paths = glob(os.path.join(extract_path, '**/*'), recursive=True)

# Filter the list to include only files
i_paths = [path for path in all_paths if os.path.isfile(path)]

print("Total images:", len(i_paths))

Total images: 12637


In [ ]:
from wolta.visual_tools import get_extensions, dataset_size_same, dataset_ratio_same, examine_sizes

get_extensions(i_paths)
dataset_size_same(i_paths)
dataset_ratio_same(i_paths)
examine_sizes(i_paths)

temp_img = cv2.imread(i_paths[0])
height, width = temp_img.shape[:2]
print('ratio: {}'.format((width/height)))


Average Width x Height: 102.64532721373743 x 103.48642874099866
Biggest width is 1836 and the smallest width is 100
Biggest height is 2314 and the smallest height is 100
ratio: 1.0


In [ ]:
raw_dir = '/content/isl_dataset/raw'
os.makedirs(raw_dir, exist_ok=True)

# Define d_paths by listing directories within extract_path, excluding 'raw' and 'data'
extract_path = '/content/isl_dataset'
d_paths = [os.path.join(extract_path, d) for d in os.listdir(extract_path) if os.path.isdir(os.path.join(extract_path, d)) and d not in ['raw', 'data']]


for d_path in d_paths:
    class_name = Path(d_path).name
    class_dir = os.path.join(raw_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

    # Use glob to find only files within the directory
    img_paths = glob(os.path.join(d_path, '*.*')) # Added .* to only match files with extensions
    for img_path in img_paths:
        img_name = Path(img_path).name
        img = cv2.imread(img_path)
        # Add a check to ensure img is not None before resizing
        if img is not None:
            img = cv2.resize(img, (224, 224))
            cv2.imwrite(os.path.join(class_dir, img_name), img)
        else:
            print(f"Warning: Could not read image file: {img_path}")

In [ ]:
import shutil

data_dir = '/content/isl_dataset/data'

# Remove the data directory if it exists
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)

dir_split(raw_dir, data_dir, val_size=0.2, test_size=0.2)

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

print("Train classes:", os.listdir(train_dir))

Train classes: ['o', 'd', 'v', 's', 'a', 'm', 'y', 'k', 'u', 'h', 'e', 'f', 'n', 't', 'q', 'l', 'j', 'x', 'r', 'p', 'w', 'i', 'b', 'g', 'c', 'z']


In [ ]:
yolo_model = YOLO(model='yolo11x-cls.pt')
yolo_results = yolo_model.train(data=data_dir, epochs=5, imgsz=224)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/isl_dataset/data, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=1

In [ ]:
# Torch transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

train_ds = datasets.ImageFolder(train_dir, transform=transform)
val_ds = datasets.ImageFolder(val_dir, transform=transform)
test_ds = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=len(train_ds.classes),
    ignore_mismatched_sizes=True
)
vit_model.to(device)
optimizer = Adam(vit_model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([26]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([26, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
epochs = 5

for epoch in range(epochs):
    vit_model.train()
    train_loss, correct = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vit_model(images)
        loss = criterion(outputs.logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs.logits, 1)
        correct += (preds == labels).sum().item()

    train_acc = 100 * correct / len(train_ds)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Accuracy: {train_acc:.2f}%")

Epoch 1/5 | Loss: 144.6501 | Accuracy: 86.42%
Epoch 2/5 | Loss: 5.2283 | Accuracy: 99.54%
Epoch 3/5 | Loss: 5.3551 | Accuracy: 99.50%
Epoch 4/5 | Loss: 1.7183 | Accuracy: 99.83%
Epoch 5/5 | Loss: 0.2741 | Accuracy: 100.00%


In [ ]:
vit_model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vit_model(images)
        _, preds = torch.max(outputs.logits, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

acc = 100 * correct / total
print(f"ViT Test Accuracy: {acc:.2f}%")

ViT Test Accuracy: 99.72%


In [ ]:
train_ds_tf = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=(224,224), batch_size=16, shuffle=True
)
val_ds_tf = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir, image_size=(224,224), batch_size=16, shuffle=True
)
test_ds_tf = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=(224,224), batch_size=16, shuffle=False
)

num_classes = len(train_ds_tf.class_names)
print("Classes:", train_ds_tf.class_names)


Found 7593 files belonging to 26 classes.
Found 2522 files belonging to 26 classes.
Found 2522 files belonging to 26 classes.
Classes: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
def get_model(c16, c32, c64, d128, d64, d32, num_classes):
    if c16==0 and c32==0 and c64==0: c64=1
    if d128==0 and d64==0 and d32==0: d128=1

    model = Sequential([layers.Input(shape=[224,224,3]), layers.Rescaling(1./255)])

    for _ in range(c16):
        model.add(layers.Conv2D(16,3,padding='same',activation='relu'))
        model.add(layers.MaxPooling2D())
    for _ in range(c32):
        model.add(layers.Conv2D(32,3,padding='same',activation='relu'))
        model.add(layers.MaxPooling2D())
    for _ in range(c64):
        model.add(layers.Conv2D(64,3,padding='same',activation='relu'))
        model.add(layers.MaxPooling2D())

    model.add(layers.Flatten())

    for _ in range(d128): model.add(layers.Dense(128,activation='relu'))
    for _ in range(d64): model.add(layers.Dense(64,activation='relu'))
    for _ in range(d32): model.add(layers.Dense(32,activation='relu'))

    model.add(layers.Dense(num_classes))
    return model

model_cnn = get_model(1,1,1,1,0,0,num_classes)
model_cnn.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = model_cnn.fit(train_ds_tf, validation_data=val_ds_tf, epochs=5)
loss, acc = model_cnn.evaluate(test_ds_tf)
print(f"CNN Test Accuracy: {acc*100:.2f}%")

Epoch 1/5
475/475 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.2529 - loss: 2.6138 - val_accuracy: 0.8204 - val_loss: 0.6050
Epoch 2/5
475/475 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8714 - loss: 0.4246 - val_accuracy: 0.9175 - val_loss: 0.2964
Epoch 3/5
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9713 - loss: 0.0967 - val_accuracy: 0.9266 - val_loss: 0.2902
Epoch 4/5
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9788 - loss: 0.0739 - val_accuracy: 0.9274 - val_loss: 0.3242
Epoch 5/5
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9828 - loss: 0.0509 - val_accuracy: 0.9266 - val_loss: 0.3328
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9257 - loss: 0.3409
CNN Test Accuracy: 91.83%


Using labels from train_ds_tf.class_names (TF).
Using labels from train_ds (torch ImageFolder).


<IPython.core.display.Javascript object>

Webcam UI injected. Use the 'Start Webcam (Predict)' button that appears above this cell to begin streaming.
Predictions (TF and ViT) will be printed below as frames are processed.
[2025-09-21 16:41:17] Final -> t (score=0.590) | TF: t(0.590) | ViT: l(0.241)
[2025-09-21 16:41:18] Final -> q (score=0.850) | TF: t(0.592) | ViT: q(0.850)
[2025-09-21 16:41:19] Final -> q (score=0.940) | TF: t(0.589) | ViT: q(0.940)
[2025-09-21 16:41:20] Final -> t (score=0.590) | TF: t(0.590) | ViT: k(0.450)
[2025-09-21 16:41:20] Final -> t (score=0.584) | TF: t(0.584) | ViT: q(0.569)
[2025-09-21 16:41:21] Final -> q (score=0.864) | TF: t(0.585) | ViT: q(0.864)
[2025-09-21 16:41:22] Final -> q (score=0.938) | TF: t(0.584) | ViT: q(0.938)
[2025-09-21 16:41:23] Final -> t (score=0.589) | TF: t(0.589) | ViT: l(0.198)
[2025-09-21 16:41:23] Final -> t (score=0.589) | TF: t(0.589) | ViT: l(0.155)
[2025-09-21 16:41:23] Final -> t (score=0.589) | TF: t(0.589) | ViT: i(0.159)
[2025-09-21 16:41:24] Final -> t (score

In [ ]:
# ------------------ Webcam live ISL overlay ------------------
from google.colab import output
from base64 import b64decode
import io
import PIL.Image
import numpy as np
import torch
import torchvision.transforms as T
import IPython.display as display
import tensorflow as tf

# --- Transforms for ViT ---
vit_transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# --- Labels ---
tf_labels = train_ds_tf.class_names  # TensorFlow CNN labels
vit_labels = train_ds.classes  # ViT labels

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_model.to(device)
vit_model.eval()  # Make sure model is in eval mode

# --- Prediction callback ---
def _process_frame_overlay(b64_img):
    try:
        img_bytes = b64decode(b64_img.split(',')[1])
        pil_img = PIL.Image.open(io.BytesIO(img_bytes)).convert('RGB')

        # TensorFlow CNN
        tf_input = np.expand_dims(np.array(pil_img.resize((224,224)))/255.0, axis=0)
        tf_logits = model_cnn.predict(tf_input, verbose=0) # Corrected model name to model_cnn
        tf_idx = int(np.argmax(tf_logits, axis=1)[0])
        tf_label = tf_labels[tf_idx]
        tf_conf = float(np.max(tf.nn.softmax(tf_logits).numpy()))

        # ViT PyTorch
        vit_img = vit_transform(pil_img).unsqueeze(0).to(device)
        with torch.no_grad():
            out = vit_model(vit_img)
            probs = torch.softmax(out.logits, dim=1).cpu().numpy()[0]
            vit_idx = int(np.argmax(probs))
            vit_label = vit_labels[vit_idx]
            vit_conf = float(probs[vit_idx])

        # Choose final label (higher confidence)
        final_label = tf_label if tf_conf >= vit_conf else vit_label
        return str(final_label)
    except Exception as e: # Added exception handling for debugging
        print("Prediction overlay error:", e)
        return "Error"

output.register_callback('notebook.overlayPredict', _process_frame_overlay)

# ---------------------- Inject JS for webcam streaming with overlay ----------------------
js_overlay = """
const streamerOverlayId = 'colab-isl-streamer-overlay';
if (!document.getElementById(streamerOverlayId)) {
  const container = document.createElement('div');
  container.id = streamerOverlayId;
  container.style = 'margin:10px 0;';

  const startBtn = document.createElement('button');
  startBtn.textContent = 'Start Webcam (Overlay Predict)';
  startBtn.style = 'margin-right:10px;';
  const stopBtn = document.createElement('button');
  stopBtn.textContent = 'Stop Webcam Overlay';
  stopBtn.disabled = true;
  stopBtn.style = 'margin-right:10px;';

  const statusSpan = document.createElement('span');
  statusSpan.textContent = 'Status: idle';
  statusSpan.style = 'margin-left:10px; font-weight:600;';

  const video = document.createElement('video');
  video.autoplay = true;
  video.playsInline = true;
  video.style = 'display:block; margin-top:10px; max-width:480px; border:1px solid #ccc;';

  const overlay = document.createElement('div');
  overlay.style = `
    position: absolute;
    top: 0;
    left: 0;
    background-color: rgba(255, 255, 255, 0.7);
    color: black;
    font-size: 24px;
    padding: 5px;
    z-index: 10;
  `;
  const overlayText = document.createElement('span');
  overlay.appendChild(overlayText);

  const videoContainer = document.createElement('div');
  videoContainer.style = 'position: relative; display: inline-block;';
  videoContainer.appendChild(video);
  videoContainer.appendChild(overlay);


  container.appendChild(startBtn);
  container.appendChild(stopBtn);
  container.appendChild(statusSpan);
  container.appendChild(videoContainer);
  document.body.appendChild(container);


  let stream = null;
  let intervalId = null;

  startBtn.onclick = async () => {
    try {
      stream = await navigator.mediaDevices.getUserMedia({video: true, audio: false});
      video.srcObject = stream;
      await video.play();
      // Adjust overlay size and position after video starts
      overlay.style.width = video.offsetWidth + 'px';
      overlay.style.height = 'auto'; // or set a fixed height
      overlay.style.top = (video.offsetTop + video.offsetHeight - overlay.offsetHeight) + 'px';
      overlay.style.left = video.offsetLeft + 'px';
      overlayText.textContent = 'Loading...'; // Initial text

      statusSpan.textContent = 'Status: streaming';
      startBtn.disabled = true;
      stopBtn.disabled = false;


      const canvas = document.createElement('canvas');
      const ctx = canvas.getContext('2d');

      intervalId = setInterval(() => {
        try {
          if (!video.videoWidth || !video.videoHeight) return; // Avoid processing before video is ready

          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
          const data = canvas.toDataURL('image/jpeg', 0.6);

          google.colab.kernel.invokeFunction('notebook.overlayPredict', [data], {})
            .then(result => {
              overlayText.textContent = result.data[0]; // Update overlay with prediction
               // Adjust overlay vertical position again if necessary
              overlay.style.top = (video.offsetTop + video.offsetHeight - overlay.offsetHeight) + 'px';
            });

        } catch(err) {
          console.log('overlay frame send error', err);
        }
      }, 700); // one frame every 700ms
    } catch (err) {
      statusSpan.textContent = 'Status: permission denied / error';
      console.error(err);
    }
  };

  stopBtn.onclick = () => {
    if (intervalId) {
      clearInterval(intervalId);
      intervalId = null;
    }
    if (stream) {
      stream.getTracks().forEach(t => t.stop());
      stream = null;
    }
    video.srcObject = null;
    statusSpan.textContent = 'Status: stopped';
    startBtn.disabled = false;
    stopBtn.disabled = true;
    overlayText.textContent = ''; // Clear overlay text on stop
  };
}
"""

display.display(display.Javascript(js_overlay))
print("Webcam Overlay UI injected. Use the 'Start Webcam (Overlay Predict)' button that appears above this cell.")
print("Predictions (TF and ViT combined) will be overlaid on the video stream.")

<IPython.core.display.Javascript object>

Webcam Overlay UI injected. Use the 'Start Webcam (Overlay Predict)' button that appears above this cell.
Predictions (TF and ViT combined) will be overlaid on the video stream.


In [ ]:
js_code = r"""
(async function() {
  const containerId = 'isl-overlay-streamer';
  if (document.getElementById(containerId)) return;
  const container = document.createElement('div');
  container.id = containerId;
  container.style = 'margin:10px 0; position: relative;';

  const startBtn = document.createElement('button');
  startBtn.textContent = 'Start Webcam (Overlay)';
  startBtn.style = 'margin-right:10px;';
  const stopBtn = document.createElement('button');
  stopBtn.textContent = 'Stop Webcam';
  stopBtn.disabled = true;
  stopBtn.style = 'margin-right:10px;';

  const statusSpan = document.createElement('span');
  statusSpan.textContent = 'Status: idle';
  statusSpan.style = 'margin-left:10px; font-weight:600;';

  const video = document.createElement('video');
  video.autoplay = true;
  video.playsInline = true;
  video.style = 'display:block; margin-top:10px; max-width:480px; border:1px solid #ccc;';

  const overlay = document.createElement('div');
  overlay.style.position = 'absolute';
  overlay.style.top = '20px';
  overlay.style.left = '20px';
  overlay.style.color = 'red';
  overlay.style.fontSize = '60px';
  overlay.style.fontWeight = 'bold';
  overlay.style.textShadow = '2px 2px black';
  overlay.innerText = '';

  container.appendChild(startBtn);
  container.appendChild(stopBtn);
  container.appendChild(statusSpan);
  container.appendChild(video);
  container.appendChild(overlay);
  document.body.appendChild(container);

  let stream=null, intervalId=null;

  startBtn.onclick = async () => {
    try {
      stream = await navigator.mediaDevices.getUserMedia({video:true});
      video.srcObject = stream;
      statusSpan.textContent='Status: streaming';
      startBtn.disabled=true;
      stopBtn.disabled=false;

      const canvas = document.createElement('canvas');
      const ctx = canvas.getContext('2d');

      intervalId = setInterval(async () => {
        canvas.width = video.videoWidth || 640;
        canvas.height = video.videoHeight || 480;
        ctx.drawImage(video,0,0,canvas.width,canvas.height);
        const data = canvas.toDataURL('image/jpeg',0.6);

        // Call Python callback and get prediction
        const result = await google.colab.kernel.invokeFunction('notebook.overlayPredict',[data],{});
        overlay.innerText = result.value;  // Display predicted letter
      }, 700);
    } catch(err){ statusSpan.textContent='Status: error'; console.error(err);}
  };

  stopBtn.onclick = () => {
    if(intervalId){ clearInterval(intervalId); intervalId=null; }
    if(stream){ stream.getTracks().forEach(t=>t.stop()); stream=null; }
    video.srcObject=null;
    overlay.innerText='';
    statusSpan.textContent='Status: stopped';
    startBtn.disabled=false; stopBtn.disabled=true;
  };
})();
"""
display.display(display.Javascript(js_code))
print("Webcam overlay UI injected. Click 'Start Webcam (Overlay)' to begin live ISL recognition.")


<IPython.core.display.Javascript object>

Webcam overlay UI injected. Click 'Start Webcam (Overlay)' to begin live ISL recognition.
